In [13]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [14]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [15]:
import pandas as pd

file_name = "https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/new_york_hotels.csv"
df = pd.read_csv(file_name, encoding = "ISO-8859-1")
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [16]:
df.shape

(1631, 11)

In [17]:
%%timeit
# This is a built in magic ipython command that provides timing for code executed in the cell
# Run the haversine looping function
df['distance'] = haversine_looping(df)

417 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit

# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

141 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
df['distance'].head()

0    139.607190
1    139.746898
2    142.191050
3    137.275546
4    139.684583
Name: distance, dtype: float64

In [22]:
%%timeit

df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']),axis=1)

41.8 ms ± 968 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit

# Vectorized implementation of Haversine applied on NumPy arrays (note we use .values to access the numpy series)
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

177 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
